# Exercise - Decision Tree

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [17]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [18]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("/Users/ayushitiwari/Downloads/Decision Tree Shared Files/healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [21]:
train_y = train['readmitted']
test_ = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [22]:
def calculate_inpatient_to_outpatient_ratio(df):
    # Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()

    # Calculate the ratio with division by zero handling. Replace 0/0 with 0.
    df1['inpatient_to_outpatient_ratio'] = df1.apply(lambda x: x['number_inpatient'] / x['number_outpatient']
                                                     if x['number_outpatient'] > 0 else 0, axis=1)
    
    # Replace infinity values with a defined value, e.g., 1 or another placeholder for cases of division by zero (other than 0/0)
    df1['inpatient_to_outpatient_ratio'].replace(np.inf, 1, inplace=True)
    
    return df1[['inpatient_to_outpatient_ratio']]

In [23]:
#Let's test the new function:

# Send the train set to the function we created
calculate_inpatient_to_outpatient_ratio(train)

,inpatient_to_outpatient_ratio
4166,0.0
5546,0.0
2957,0.0
6329,0.0
565,0.0
...,...
5734,0.0
5191,0.0
5390,0.0
860,0.0


##  Identify the numeric, binary, and categorical columns

In [24]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [25]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

# Wrap your function with FunctionTransformer
#ratio_transformer = FunctionTransformer(calculate_inpatient_to_outpatient_ratio)

In [26]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [27]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [28]:
feat_eng_columns = ['number_inpatient', 'number_outpatient']

# Pipeline

In [29]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [30]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [31]:
# Create a pipeline for the transformed column here

my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(calculate_inpatient_to_outpatient_ratio)),
                               ('scaler', StandardScaler())])

In [33]:
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_columns),
    ('cat', categorical_transformer, categorical_columns),
    ('ratio', RatioTransformer('number_inpatient', 'number_outpatient'), ['number_inpatient', 'number_outpatient'])
], remainder='passthrough')


#passtrough is an optional step. You don't have to use it.

NameError: name 'RatioTransformer' is not defined

# Transform: fit_transform() for TRAIN

In [ ]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

In [ ]:
train_x.shape

# Tranform: transform() for TEST

In [ ]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

In [ ]:
test_x.shape

# Calculate the Baseline

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

In [ ]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

In [ ]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

# Train a DT model

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

tree_clf.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [ ]:

#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_y, train_y_pred))


In [ ]:

#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_y, test_y_pred))


# Train another DT model (with different parameters)

In [ ]:
tree_clf2 = DecisionTreeClassifier(min_samples_leaf = 10, max_depth=6)

tree_clf2.fit(train_x, train_y)



### Calculate the accuracy

In [ ]:
#Train accuracy:
train_y_pred = tree_clf2.predict(train_x)

print(accuracy_score(train_y, train_y_pred))

In [ ]:
#Test accuracy:
test_y_pred = tree_clf2.predict(test_x)

print(accuracy_score(test_y, test_y_pred))

# Optional: try grid search